In [ ]:
# For colab
!pip uninstall tensorflow -y
!pip install tensorflow==2.3.1 tensorflow-quantum
!rm -rf quantum-gans
!git clone https://github.com/WiktorJ/quantum-gans
!cd quantum-gans; pip install .

In [1]:
from typing import Any

import sympy
import math
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import numpy as np
from cirq import GridQubit, ops
from qugans import circuits
from qugans.phase_transition import construct_hamiltonian, build_ground_state_circuit, \
    get_theta_v, get_theta_w, get_theta_r

In [2]:
generator_layers = 2
discriminator_layers = 2
data_bus_size = 3

In [21]:
gen, gs, disc, ds, ls, data_qubits, out_qubit = circuits.build_gan_circuits(generator_layers, discriminator_layers, data_bus_size)

In [22]:
real, real_symbols = build_ground_state_circuit(qubits=data_qubits)

In [23]:
pure_gen = gen.copy()
gen.append([disc])

In [24]:
pure_real = real.copy()
real.append([disc])

In [25]:
print("REAL GROUND STATE: ", pure_real)

REAL GROUND STATE:  (0, 0): ───H───@────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
               │
(0, 1): ───────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────Ry(theta1)───X───Ry(theta1)───Ry(theta0)───X───Ry(theta0)───
               │                                                                                                                                                         │                             │
(0, 2): ───────X───Z───Ry(theta2)───X────────────@───X─────────────────────────@───X─────────────────────────────────────────────────────────────────────────────────────┼─────────────────────────────┼────────────────
                                                 │                             │                               

In [26]:
print("GENERATOR:", pure_gen)

GENERATOR: (0, 2): ───Rx(g0)───Rz(g4)───ZZ────────────────────────Rx(g11)───Rz(g15)───ZZ──────────────────────────
                             │                                             │
(0, 3): ───Rx(g1)───Rz(g5)───ZZ^g8────────────ZZ───────Rx(g12)───Rz(g16)───ZZ^g19─────────────ZZ───────
                                              │                                               │
(0, 4): ───Rx(g2)───Rz(g6)────────────ZZ──────ZZ^g10───Rx(g13)───Rz(g17)─────────────ZZ───────ZZ^g21───
                                      │                                              │
(0, 5): ───Rz(l)────Rx(g3)───Rz(g7)───ZZ^g9────────────Rz(l)─────Rx(g14)───Rz(g18)───ZZ^g20────────────


In [27]:
print("DISCRIMINATOR:", disc)

DISCRIMINATOR: (0, 0): ───Rx(d0)───Rz(d5)────────────ZZ────────────────Rx(d14)───Rz(d19)─────────────ZZ────────────────
                                      │                                               │
(0, 1): ───Rz(l)────Rx(d1)───Rz(d6)───ZZ^d10───ZZ───────Rz(l)─────Rx(d15)───Rz(d20)───ZZ^d24───ZZ───────
                                               │                                               │
(0, 2): ───Rx(d2)───Rz(d7)───ZZ────────────────ZZ^d12───Rx(d16)───Rz(d21)───ZZ─────────────────ZZ^d26───
                             │                                              │
(0, 3): ───Rx(d3)───Rz(d8)───ZZ^d11───ZZ────────────────Rx(d17)───Rz(d22)───ZZ^d25────ZZ────────────────
                                      │                                               │
(0, 4): ───Rx(d4)───Rz(d9)────────────ZZ^d13────────────Rx(d18)───Rz(d23)─────────────ZZ^d27────────────


In [28]:
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 21) + \
                   np.random.normal(scale=eps, size=(22,))
init_disc_weights = np.random.normal(size=(28,))

gen_weights = tf.Variable(init_gen_weights, dtype=tf.float32)
disc_weights = tf.Variable(init_disc_weights, dtype=tf.float32)

In [29]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        return max(10 * math.e ** - (step / (self.warmup_steps / math.log(100))), 0.1)

In [30]:
learning_rate = CustomSchedule()

opt = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [31]:
global_g = 0

In [32]:
def map_to_radians(val, low=-1, high=1):
    span = high - low
    norm_val = float(val - low) / float(span)
    return norm_val * math.pi

In [33]:
class PhaseTransitionFinalStateSimulator(cirq.Simulator):
    def simulate(self, program: 'cirq.Circuit', param_resolver: 'study.ParamResolverOrSimilarType' = None,
                 qubit_order: ops.QubitOrderOrList = ops.QubitOrder.DEFAULT,
                 initial_state: Any = None) -> 'SimulationTrialResult':
        g = global_g
        gzz = 2 * (1 - g ** 2)
        gx = (1 + g) ** 2
        gzxz = (g - 1) ** 2

        H = construct_hamiltonian(data_bus_size, gzz, gx, gzxz)

        lam, V = np.linalg.eigh(H)

        # ground state wavefunction
        psi = V[:, 0] / np.linalg.norm(V[:, 0])
        psi = np.kron([1, 0], np.kron([1, 0], psi))
        return super().simulate(program, param_resolver, qubit_order, psi)


class PhaseTransitionFinalStateSimulatorGen(cirq.Simulator):
    def simulate(self, program: 'cirq.Circuit', param_resolver: 'study.ParamResolverOrSimilarType' = None,
                 qubit_order: ops.QubitOrderOrList = ops.QubitOrder.DEFAULT,
                 initial_state: Any = None) -> 'SimulationTrialResult':
        g = global_g
        label = [1, 0] if g > 0 else [0, 1]
        initial_state = [1, 0]
        for _ in range(l - 1):
            initial_state = np.kron([1, 0], initial_state)
        initial_state = np.kron([1, 0], np.kron(label, np.kron(initial_state, label)))
        return super().simulate(program, param_resolver, qubit_order, initial_state)





In [34]:
def real_disc_circuit_eval_mocked(disc_weights):
    # cirq.Simulator().simulate(real)
    full_weights = tf.keras.layers.Concatenate(axis=0)([
        disc_weights,
        np.array([map_to_radians(global_g)], dtype=np.float32)
    ])

    return tfq.layers.Expectation(backend=PhaseTransitionFinalStateSimulator())([disc],
                                                                                symbol_names=ds + (ls,),
                                                                                symbol_values=tf.reshape(full_weights, (
                                                                                    1, full_weights.shape[0])),
                                                                                operators=[cirq.Z(out_qubit)])


def real_disc_circuit_eval(disc_weights):
    # cirq.Simulator().simulate(real)
    full_weights = tf.keras.layers.Concatenate(axis=0)([
        np.array([get_theta_v(global_g), get_theta_w(global_g), get_theta_r(global_g)], dtype=np.float32),
        disc_weights,
        np.array([map_to_radians(global_g)], dtype=np.float32)
    ])

    return tfq.layers.Expectation()([real],
                                    symbol_names=real_symbols + ds + (ls,),
                                    symbol_values=tf.reshape(full_weights, (
                                        1, full_weights.shape[0])),
                                    operators=[cirq.Z(out_qubit)])


def gen_disc_circuit_eval(gen_weights, disc_weights):
    full_weights = tf.keras.layers.Concatenate(axis=0)([
        disc_weights,
        gen_weights,
        np.array([map_to_radians(global_g)], dtype=np.float32)
    ])
    full_weights = tf.reshape(full_weights, (1, full_weights.shape[0]))
    return tfq.layers.Expectation()([gen],
                                    symbol_names=ds + gs + (ls,),
                                    symbol_values=full_weights,
                                    operators=[cirq.Z(out_qubit)])


def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit_eval(disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit_eval(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights):
    return -prob_fake_true(gen_weights, disc_weights)


In [35]:
def train():
    cost = lambda: disc_cost(disc_weights)
    cost_gen = lambda: gen_cost(gen_weights)
    for epoch in range(20):
        for step in range(20):
            opt.minimize(cost, disc_weights)
            # if step % 5 == 0:
        cost_val = cost().numpy()
        print("Epoch {}: cost = {}".format(epoch, cost_val))

        ##############################################################################
        # At the discriminator’s optimum, the probability for the discriminator to
        # correctly classify the real data should be close to one.

        print("Prob(real classified as real): ", prob_real_true(disc_weights).numpy())

        ##############################################################################
        # For comparison, we check how the discriminator classifies the
        # generator’s (still unoptimized) fake data:

        print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

        ##############################################################################
        # In the adversarial game we now have to train the generator to better
        # fool the discriminator. For this demo, we only perform one stage of the
        # game. For more complex models, we would continue training the models in an
        # alternating fashion until we reach the optimum point of the two-player
        # adversarial game.

        for step in range(20):
            opt.minimize(cost_gen, gen_weights)
            # if step % 5 == 0:
        cost_val = cost_gen().numpy()
        print("Epoch {}: cost = {}".format(epoch, cost_val))

        ##############################################################################
        # At the optimum of the generator, the probability for the discriminator
        # to be fooled should be close to 1.

        print("Prob(fake classified as real): ", prob_fake_true(gen_weights, disc_weights).numpy())

        ##############################################################################
        # At the joint optimum the discriminator cost will be close to zero,
        # indicating that the discriminator assigns equal probability to both real and
        # generated data.

        print("Discriminator cost: ", disc_cost(disc_weights).numpy())
        print("Generator weights:", gen_weights)
        print("Discriminator weights", disc_weights)



In [36]:
def get_ground_state_for_g(g):
    gzz = 2 * (1 - g ** 2)
    gx = (1 + g) ** 2
    gzxz = (g - 1) ** 2

    H = construct_hamiltonian(data_bus_size, gzz, gx, gzxz)

    lam, V = np.linalg.eigh(H)
    # ground state wavefunction
    return V[:, 0] / np.linalg.norm(V[:, 0])

In [37]:
class PhaseTransitionFinalStateSimulatorPureGen(cirq.Simulator):
    def simulate(self, program: 'cirq.Circuit', param_resolver: 'study.ParamResolverOrSimilarType' = None,
                 qubit_order: ops.QubitOrderOrList = ops.QubitOrder.DEFAULT,
                 initial_state: Any = None) -> 'SimulationTrialResult':
        label = [1, 0]
        initial_state = [1, 0]
        for _ in range(l - 1):
            initial_state = np.kron([1, 0], initial_state)
        initial_state = np.kron(initial_state, label)
        return super().simulate(program, param_resolver, qubit_order, initial_state)

In [ ]:
train()

In [19]:
g = 0
ground_state = get_ground_state_for_g(g)
rad = map_to_radians(g)

trained_disc_weights = tf.Variable(np.array([-245.17047   ,  -95.07969   , -342.84647   , -422.7227    ,
       -183.86722   , -279.45145   , -208.48354   , -126.94892   ,
         99.149605  , -276.1524    ,  -82.480415  , -197.35036   ,
       -115.24699   , -111.50772   , -238.86653   , -150.12883   ,
       -271.65726   , -414.5756    , -183.14653   ,   97.29306   ,
       -120.5858    ,   -4.429718  ,  127.67174   ,  118.280525  ,
        305.74625   ,   -0.44963428,   17.841654  ,   93.42311   ]), dtype=tf.float32)

trained_gen_weights = tf.Variable(np.array([-44.918114 , -33.19301  , -65.66357  ,   7.7173038, -21.041788 ,
       -65.616486 , -47.297474 ,  62.851395 , -18.168066 , -21.922869 ,
        52.146767 , -70.66162  , -11.20047  ,  13.477829 , -24.01582  ,
        82.88633  , -82.02305  , -73.62559  ,  11.767767 ,  -3.0552819,
       -37.224037 ,  -1.1020341, rad]), dtype=tf.float32)


state_vector = tfq.layers.State()(pure_gen, symbol_names=gs + (ls,), symbol_values=tf.reshape(trained_gen_weights, (1, trained_gen_weights.shape[0]))).values.numpy()#%%

traced_out_state = cirq.wavefunction_partial_trace_as_mixture(state_vector, [0,1,2])

<ipython-input-19-6c30ee798a71>:22: DeprecationWarning: wavefunction_partial_trace_as_mixture was used but is deprecated.
It will be removed in cirq v0.10.0.
Use `cirq.partial_trace_of_state_vector_as_mixture` instead.

  traced_out_state = cirq.wavefunction_partial_trace_as_mixture(state_vector, [0,1,2])


In [20]:
prob, highest_prob_state = max(traced_out_state)
print(f"Max prob state has prob: {prob}")

Max prob state has prob: 0.9586108922958374


In [21]:
np.arccos(cirq.fidelity(highest_prob_state, get_ground_state_for_g(1)))

1.5001486857260862

In [ ]:
cirq.fidelity(highest_prob_state, get_ground_state_for_g(1))